# Kinematic generator
This notebook converts a Denavit Hartenberg table to its direct kinematic symbolic calculation and its jacobian

In [114]:
from cmath import pi
import numpy as np
import sympy
from sympy.abc import x,y,z,a,d

# import sys
# sys.set_int_max_str_digits(0)

## Joints
This classes define a single joint in is symbolic representation and its type

In [115]:
from enum import Enum

class JointType(Enum):
    ROTATIONAL = 0
    PRISMATIC = 1

class Joint:
    def __init__(self, symbol: sympy.Symbol, jointType: JointType):
        self.type = jointType
        self.symbol = symbol
        self.value = 0.0

## Denavit Hartenberg
Symbolic Denavit table 

In [116]:
class Denavit:
    def __init__(self):
        self.theta = sympy.Symbol('theta')
        self.alfa = sympy.Symbol('alpha') 
        self.d = sympy.Symbol('d')
        self.a = sympy.Symbol('a')

        cosT = sympy.cos(self.theta)
        sinT = sympy.sin(self.theta)
        cosA = sympy.cos(self.alfa)
        sinA = sympy.sin(self.alfa)
        
        self.denavit = sympy.Matrix([
            [cosT,  -sinT*cosA, sinT*sinA,  self.a*cosT ],
            [sinT,  cosT*cosA,  -cosT*sinA, self.a*sinT ],
            [0,     sinA,       cosA,       self.d      ],
            [0,     0,          0,          1           ]
        ])

Denavit().denavit

Matrix([
[cos(theta), -sin(theta)*cos(alpha),  sin(alpha)*sin(theta), a*cos(theta)],
[sin(theta),  cos(alpha)*cos(theta), -sin(alpha)*cos(theta), a*sin(theta)],
[         0,             sin(alpha),             cos(alpha),            d],
[         0,                      0,                      0,            1]])

## Denavit row of the table
This class sets a single row of the Denavit Hartenberg table and computes the transformation matrixes, both in symbolic and lambda representations

In [117]:
class DenavitRow(Denavit):
    def __init__(self, dh_theta:float, dh_d:float, dh_a:float, dh_alfa:float, joint:Joint = None):
        super().__init__()
        self.dhParams = (dh_theta,dh_d,dh_a,dh_alfa)
        self.joint = joint

        if joint is not None:
            if joint.type is JointType.ROTATIONAL:
                T = self.denavit.subs(self.theta, self.theta+joint.symbol)
            elif joint.type is JointType.PRISMATIC:
                T = self.denavit.subs(self.d, self.d+joint.symbol)
            else:
                raise ValueError("Invalid joint type")
        else:
            T = self.denavit
        
        T = T.subs(self.theta,dh_theta)
        T = T.subs(self.d,dh_d)
        T = T.subs(self.a,dh_a)
        T = T.subs(self.alfa,dh_alfa)
        self.TransformSym = T

    def get(self):
        return (self.theta,self.d, self.a, self.alfa, self.joint.value)

    def eval(self, jointVal:float):
        self.joint.value = jointVal
        return self.Transform(*self.get())
    
    def getRotationSym(self):
        return self.TransformSym[0:3,0:3]
    
    def getTranslationSym(self):
        return self.TransformSym[0:3,3]

## Denavit Direct kinematics
This class takes an ordered list of the Denavit rows and computes the total kinematics in symbolics and lambda

In [118]:
class DenavitDK:
    def __init__(self,denavitRows) -> None:
        self.directTransformSym = sympy.Matrix(np.eye(4))
        self.jointsSym = []
        # Compute direct kinematis and record joint symbols
        for T in denavitRows:
            if T.joint is not None:
                self.jointsSym.append(T.joint.symbol)
            self.directTransformSym = self.directTransformSym*T.TransformSym
        self.directTransformSym = sympy.simplify(self.directTransformSym)
        # Set joints number
        self.jointsNum = len(self.jointsSym)
        # Set joints array
        self.jointsSym = sympy.Matrix([q for q in self.jointsSym])
        # Set lambda for direct transform
        self.directTransform = sympy.lambdify(self.jointsSym,self.directTransformSym)
        # Calculate jacobian
        self._jacobian(denavitRows)
        # Calculate inverse jacobian
        self._invJacobian()
    
    def eval(self, jointVal:list):
        return self.directTransform(*jointVal)

    def getRotationSym(self):
        return self.directTransformSym[0:3,0:3]
    
    def getTranslationSym(self):
        return self.directTransformSym[0:3,3]
    
    def _jacobian(self, denavitRows):
        Transform = sympy.eye(4)
        z_vector = sympy.Matrix([[0],[0],[1]])
        J = []
        for T in denavitRows:
            if T.joint.type is JointType.PRISMATIC:
                translation = Transform[0:3,0:3]*z_vector
                J_row = sympy.Matrix([translation,sympy.zeros(3,1)]).T
            elif T.joint.type is JointType.ROTATIONAL:
                rotation    = Transform[0:3,0:3]*z_vector
                translation = rotation.cross(self.getTranslationSym()-Transform[0:3,3])
                J_row = sympy.Matrix([translation,rotation]).T
            else:
                raise ValueError("Invalid joint type")
            Transform = Transform*T.TransformSym

            J.append(J_row)
        self.jacobian = sympy.Matrix(J).T
        return self.jacobian
    
    def _invJacobian(self):
        M = sympy.MatrixSymbol('M',6,self.jointsNum)
        if 6 == self.jointsNum:
            pinvM = M**-1
        elif 6 > self.jointsNum:
            pinvM = M.T * (M*M.T)**-1
        else: # 6 < self.jointsNum
            pinvM = (M.T*M)**-1 * M.T
        self.invJacobian = pinvM.subs(M,self.jacobian)
        return self.invJacobian


# Examples

## 3 DOF Cartesian 
A 3 DOF Cartesian robot with 3 prismatic joints from [The Ultimate Guide to Jacobian Matrices for Robotics](https://automaticaddison.com/the-ultimate-guide-to-jacobian-matrices-for-robotics/https://automaticaddison.com/the-ultimate-guide-to-jacobian-matrices-for-robotics/)

In [119]:
T1 = DenavitRow(pi/2,   0, 0, pi/2,  Joint(sympy.Symbol('q_1'),JointType.PRISMATIC))
T2 = DenavitRow(pi/2,   0, 0, -pi/2, Joint(sympy.Symbol('q_2'),JointType.PRISMATIC))
T3 = DenavitRow(0,      0, 0, 0,     Joint(sympy.Symbol('q_3'),JointType.PRISMATIC))

T_total = DenavitDK((T1,T2,T3))
T_total.directTransformSym

Matrix([
[-6.12323399573677e-17,                  -1.0,                    0,                                                       1.0*q_2],
[ 6.12323399573677e-17,                     0,                 -1.0,                           -6.12323399573677e-17*q_2 - 1.0*q_3],
[                  1.0, -6.12323399573677e-17, 6.12323399573677e-17, 1.0*q_1 + 6.12323399573677e-17*q_2 + 6.12323399573677e-17*q_3],
[                    0,                     0,                    0,                                                           1.0]])

In [120]:
T_total.jacobian

Matrix([
[0,                   1.0,                    0],
[0, -6.12323399573677e-17,                 -1.0],
[1,  6.12323399573677e-17, 6.12323399573677e-17],
[0,                     0,                    0],
[0,                     0,                    0],
[0,                     0,                    0]])

In [121]:
cartesian = T_total.jacobian * T_total.jointsSym
cartesian

Matrix([
[                                                  1.0*q_2],
[                      -6.12323399573677e-17*q_2 - 1.0*q_3],
[q_1 + 6.12323399573677e-17*q_2 + 6.12323399573677e-17*q_3],
[                                                        0],
[                                                        0],
[                                                        0]])

In [122]:
from sympy.abc import x,y,z
ox,oy,oz = sympy.symbols('omega_x omega_y omega_z')
cartesian = sympy.Matrix([
    x, y, z, ox, oy, oz
])

joints_spd = T_total.jacobian.T*cartesian
joints_spd

Matrix([
[                                                      z],
[1.0*x - 6.12323399573677e-17*y + 6.12323399573677e-17*z],
[                        -1.0*y + 6.12323399573677e-17*z]])

## 3 DOF Articulated Robot
A 3 DOF Articulated robot with 3 rotational joints from [The Ultimate Guide to Jacobian Matrices for Robotics](https://automaticaddison.com/the-ultimate-guide-to-jacobian-matrices-for-robotics/https://automaticaddison.com/the-ultimate-guide-to-jacobian-matrices-for-robotics/)

In [123]:
T1 = DenavitRow(0,  1, 0,   pi/2, Joint(sympy.Symbol('q_1'),JointType.ROTATIONAL))
T2 = DenavitRow(0,  0,  1,  0,    Joint(sympy.Symbol('q_2'),JointType.ROTATIONAL))
T3 = DenavitRow(0,  0,  1,  0,    Joint(sympy.Symbol('q_3'),JointType.ROTATIONAL))

T_total = DenavitDK((T1,T2,T3))
T_total.directTransformSym

Matrix([
[1.0*cos(q_1)*cos(q_2 + q_3), -1.0*sin(q_2 + q_3)*cos(q_1),         1.0*sin(q_1), -6.12323399573677e-17*sin(q_1)*sin(q_2) + 1.0*cos(q_1)*cos(q_2) + 1.0*cos(q_1)*cos(q_2 + q_3)],
[1.0*sin(q_1)*cos(q_2 + q_3), -1.0*sin(q_1)*sin(q_2 + q_3),        -1.0*cos(q_1),  1.0*sin(q_1)*cos(q_2) + 1.0*sin(q_1)*cos(q_2 + q_3) + 6.12323399573677e-17*sin(q_2)*cos(q_1)],
[         1.0*sin(q_2 + q_3),           1.0*cos(q_2 + q_3), 6.12323399573677e-17,                                                       1.0*sin(q_2) + 1.0*sin(q_2 + q_3) + 1.0],
[                          0,                            0,                    0,                                                                                           1.0]])

In [124]:
T_total.jacobian

Matrix([
[-1.0*sin(q_1)*cos(q_2) - 1.0*sin(q_1)*cos(q_2 + q_3) - 6.12323399573677e-17*sin(q_2)*cos(q_1),                                         -1.0*(1.0*sin(q_2) + 1.0*sin(q_2 + q_3))*cos(q_1) - 6.12323399573677e-17*sin(q_1)*cos(q_2) - 6.12323399573677e-17*sin(q_1)*cos(q_2 + q_3) - 3.74939945665464e-33*sin(q_2)*cos(q_1), -6.12323399573677e-17*sin(q_1)*cos(q_2 + q_3) - 1.0*sin(q_2 + q_3)*cos(q_1)],
[-6.12323399573677e-17*sin(q_1)*sin(q_2) + 1.0*cos(q_1)*cos(q_2) + 1.0*cos(q_1)*cos(q_2 + q_3),                                         -1.0*(1.0*sin(q_2) + 1.0*sin(q_2 + q_3))*sin(q_1) - 3.74939945665464e-33*sin(q_1)*sin(q_2) + 6.12323399573677e-17*cos(q_1)*cos(q_2) + 6.12323399573677e-17*cos(q_1)*cos(q_2 + q_3), -1.0*sin(q_1)*sin(q_2 + q_3) + 6.12323399573677e-17*cos(q_1)*cos(q_2 + q_3)],
[                                                                                            0, 1.0*(-6.12323399573677e-17*sin(q_1)*sin(q_2) + 1.0*cos(q_1)*cos(q_2) + 1.0*cos(q_1)*cos(q_2 + q_3))*cos(q_1

In [125]:
cartesian = T_total.jacobian * T_total.jointsSym
cartesian

Matrix([
[q_1*(-1.0*sin(q_1)*cos(q_2) - 1.0*sin(q_1)*cos(q_2 + q_3) - 6.12323399573677e-17*sin(q_2)*cos(q_1)) + q_2*(-1.0*(1.0*sin(q_2) + 1.0*sin(q_2 + q_3))*cos(q_1) - 6.12323399573677e-17*sin(q_1)*cos(q_2) - 6.12323399573677e-17*sin(q_1)*cos(q_2 + q_3) - 3.74939945665464e-33*sin(q_2)*cos(q_1)) + q_3*(-6.12323399573677e-17*sin(q_1)*cos(q_2 + q_3) - 1.0*sin(q_2 + q_3)*cos(q_1))],
[q_1*(-6.12323399573677e-17*sin(q_1)*sin(q_2) + 1.0*cos(q_1)*cos(q_2) + 1.0*cos(q_1)*cos(q_2 + q_3)) + q_2*(-1.0*(1.0*sin(q_2) + 1.0*sin(q_2 + q_3))*sin(q_1) - 3.74939945665464e-33*sin(q_1)*sin(q_2) + 6.12323399573677e-17*cos(q_1)*cos(q_2) + 6.12323399573677e-17*cos(q_1)*cos(q_2 + q_3)) + q_3*(-1.0*sin(q_1)*sin(q_2 + q_3) + 6.12323399573677e-17*cos(q_1)*cos(q_2 + q_3))],
[                                                                          q_2*(1.0*(-6.12323399573677e-17*sin(q_1)*sin(q_2) + 1.0*cos(q_1)*cos(q_2) + 1.0*cos(q_1)*cos(q_2 + q_3))*cos(q_1) + 1.0*(1.0*sin(q_1)*cos(q_2) + 1.0*sin(q_1)*cos(q_2 + 

In [126]:
from sympy.abc import x,y,z
ox,oy,oz = sympy.symbols('omega_x omega_y omega_z')
cartesian = sympy.Matrix([
    x, y, z, ox, oy, oz
])

joints_spd = T_total.jacobian.T*cartesian
joints_spd

Matrix([
[                                                                                                                                                                                                                                                                                                                                                                                                                                                                              omega_z + x*(-1.0*sin(q_1)*cos(q_2) - 1.0*sin(q_1)*cos(q_2 + q_3) - 6.12323399573677e-17*sin(q_2)*cos(q_1)) + y*(-6.12323399573677e-17*sin(q_1)*sin(q_2) + 1.0*cos(q_1)*cos(q_2) + 1.0*cos(q_1)*cos(q_2 + q_3))],
[1.0*omega_x*sin(q_1) - 1.0*omega_y*cos(q_1) + 6.12323399573677e-17*omega_z + x*(-1.0*(1.0*sin(q_2) + 1.0*sin(q_2 + q_3))*cos(q_1) - 6.12323399573677e-17*sin(q_1)*cos(q_2) - 6.12323399573677e-17*sin(q_1)*cos(q_2 + q_3) - 3.74939945665464e-33*sin(q_2)*cos(q_1)) + y*(-1.0*(1.0*sin(q_2) + 1.0*sin(q_2 + q_3))*sin(q_1) - 

In [127]:
joint_values = (0, 0, 0)
T_total.eval(joint_values)

array([[ 1.000000e+00, -0.000000e+00,  0.000000e+00,  2.000000e+00],
       [ 0.000000e+00, -0.000000e+00, -1.000000e+00,  0.000000e+00],
       [ 0.000000e+00,  1.000000e+00,  6.123234e-17,  1.000000e+00],
       [ 0.000000e+00,  0.000000e+00,  0.000000e+00,  1.000000e+00]])